In [ ]:
# Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from scipy.io import savemat

In [ ]:
# Change fonts and specify font size
plt.rcParams['font.family'] = 'serif'
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
FONT_SIZE = 12

In [ ]:
# Define necessary functions
# Function to save files
def save_file(data, file_name, file_directory):
    
    # Save the file as a .mat file
    savemat(os.path.join(file_directory, file_name), {'data': data})

    print(f'Saved {file_name} to {file_directory}')

In [ ]:
# Define function to renormalize the dataset
def renormalize_data(data_to_be_renormalized, training_mean, training_range):

    renormalized_data = (data_to_be_renormalized * training_range) + training_mean

    return renormalized_data

In [ ]:
# Define directories
current_directory = os.getcwd()

# Define directory for the normalized test data
normalized_data_directory = os.path.join(current_directory, '..', '..', 'data', 'normalized')

# Define directory for the trained results
trained_results_directory = os.path.join(current_directory, '..', '..', 'cnn', 'training_results')

# Define directory for the predicted results
predicted_results_directory = os.path.join(current_directory, '..', '..', 'cnn', 'prediction_results')

In [ ]:
# Load the normalized test subsets for displacement data
print('Loading the normalized test subsets for displacement data...')
normalized_test_displacement_data = np.load(os.path.join(normalized_data_directory, 'normalized_test_displacement_data.npy'))

In [ ]:
# Load the normalized test subsets for force data
print('Loading the normalized test subsets for force data...')
normalized_test_force_data = np.load(os.path.join(normalized_data_directory, 'normalized_test_force_data.npy'))

In [ ]:
# Print the shapes of the displacement and force data
print(f'The shape of displacement data is {normalized_test_displacement_data.shape[1:]}.')
print(f'The shape of force data is {normalized_test_force_data.shape[1:]}.')

In [ ]:
# Load the best trained model
print('Loading the best trained model...')
best_trained_model = tf.keras.models.load_model(os.path.join(trained_results_directory, 'best_model.h5'), compile = False)

In [ ]:
# Print model summary
print(best_trained_model.summary())

In [ ]:
# Make predictions using the best trained model
print('Making predictions using the best trained model...')
predicted_normalized_test_force_data = best_trained_model.predict(normalized_test_displacement_data)

In [ ]:
# Load the training mean and range
normalizing_force_parameters = np.load(os.path.join(normalized_data_directory, 'normalizing_force_parameters.npy'), allow_pickle=True)
force_mean = normalizing_force_parameters.item().get('force_mean')
force_range = normalizing_force_parameters.item().get('force_range')

In [ ]:
# Renormalize the target and force data
print('Renormalizing the target and force data...')
predicted_test_force_data = renormalize_data(predicted_normalized_test_force_data, force_mean, force_range)
target_test_force_data = renormalize_data(normalized_test_force_data, force_mean, force_range)

In [ ]:
# Calculate the error
numerator = np.sum(np.sum((np.abs(target_test_force_data - predicted_test_force_data))**2, axis=1), axis=1)
denominator = np.sum(np.sum((np.abs(target_test_force_data))**2, axis=1), axis=1)
error = numerator/denominator*100

In [ ]:
# Analyze the error values
# Sort the error values for Q1, Median, and Q3
sorted_error = np.sort(error)

In [ ]:
# Minimum error
min_error_location = np.where(error == sorted_error[0])
min_error = sorted_error[0]

In [ ]:
# Q1 error
q1_location = int(len(sorted_error)/4)
q1_error = sorted_error[q1_location]

In [ ]:
# Median error
median_location = int(len(sorted_error)/2)
median_error = sorted_error[median_location]

In [ ]:
# Q3 error
q3_location = int(3*len(sorted_error)/4)
q3_error = sorted_error[q3_location]

In [ ]:
# Max error
max_error_location = np.where(error == sorted_error[-1])
max_error = sorted_error[-1]

In [ ]:
# Print the summary of the error
print(f'The minimum error is {min_error:.2f}%.')
print(f'The Q1 error is {q1_error:.2f}%.')
print(f'The median error is {median_error:.2f}%.')
print(f'The Q3 error is {q3_error:.2f}%.')
print(f'The maximum error is {max_error:.2f}%.')
print(f'The mean error is {np.mean(error):.2f}%.')

In [ ]:
# Save the target and predicted force data
print('Saving the target and predicted force data...')
save_file(predicted_test_force_data, 'predicted_test_force_data.mat', predicted_results_directory)
save_file(target_test_force_data, 'target_test_force_data.mat', predicted_results_directory)